### Finance Dictionary 

Web scraping finance dictionary websites

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

url = "https://www.investopedia.com/financial-term-dictionary-4769738"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [ ]:
import json

# Dictionary to store finance terms and their corresponding links
finance_dict = {}

# Find all anchor tags with the attribute rel="nocaes"
for tag in soup.find_all('a', attrs={'rel': 'nocaes'}):
    term = tag.get_text(strip=True)  # Extract the term
    link = tag['href']  # Extract the link
    
    # Store in dictionary (assuming links are relative, prepend domain if needed)
    finance_dict[term] = link

# Save dictionary as JSON file
with open("finance_terms.json", "w", encoding='utf-8') as json_file:
    json.dump(finance_dict, json_file, indent=4, ensure_ascii=False)

print("Finance dictionary saved as finance_terms.json")

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import json

# # Function to fetch the definition from a given link
# def fetch_definition(url):
#     try:
#         response = requests.get(url, timeout=5)
#         response.raise_for_status()
#         soup = BeautifulSoup(response.text, "html.parser")

#         # Adjust the selector based on the webpage's structure
#         definition_section = soup.find("p")  # Modify this as needed
#         if definition_section:
#             return definition_section.get_text(strip=True)
#         else:
#             return "Definition not found."
#     except requests.exceptions.RequestException as e:
#         return f"Error fetching definition: {str(e)}"



# # Load the main scraped webpage
# with open("scraped_html.html", "r", encoding="utf-8") as file:
#     soup = BeautifulSoup(file, "html.parser")

# # Extract finance terms and their definitions
# finance_dict = {}

# for link in soup.find_all("a", rel="nocaes"):
#     term = link.text.strip()
#     href = link.get("href")

#     if href and href.startswith("http"):
#         definition = fetch_definition(href)
#         finance_dict[term.lower()] = definition  # Store definition instead of URL

# # Save the dictionary as JSON
# with open("finance_dictionary.json", "w", encoding="utf-8") as json_file:
#     json.dump(finance_dict, json_file, indent=4, ensure_ascii=False)

# print("Finance dictionary saved successfully!")


Dictionary code

In [ ]:
import os
from dotenv import load_dotenv
import streamlit as st
import json
from google import genai
from google.genai import types

# Set up Gemini API key
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

if api_key:
    client = genai.Client(api_key=api_key)
else:
    print("API key not found. Please set GEMINI_API_KEY in your .env file.")

# Load finance dictionary from JSON file
def load_finance_dictionary():
    with open("finance_terms.json", "r", encoding="utf-8") as file:
        return json.load(file)

def generate_simple_explanation(term):
    sys_instruct="You are a model that simplifies the financial terms for people who do not know anything about finance. Given a word, explain it in a way anyone can understand. But the meaning should be correct."

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        config=types.GenerateContentConfig(
            system_instruction=sys_instruct),
        contents=[f"The term is: {term}. Explain it in simple words."]
        )
    if response:
        return response.text
    else:
        return "I couldn't simplify this term."
    
def generate_proper_explanation(term):
    sys_instruct="You are a finance dictionary. Take any finance term and provide a formal definition of it like one given in a dictionary."

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        config=types.GenerateContentConfig(
            system_instruction=sys_instruct),
        contents=[f"The term is: {term}. Explain it like a dictionary"]
        )
    if response:
        return response.text
    else:
        return "I couldn't explain this term."

def calculate_risk_tolerance(user_age, user_investment_exp,user_income, user_education):
    # Example logic: You can expand and adjust this based on your actual calculation
    age_factor = {"18-25":10, "26-30":9, "31-40":8, "41-50":6,"51-60":4,"60+":2}
    investment_exp_factor = {"Less than 1 year": 2, "1-3 years": 0.5, "3-5 years": 7, "5+ years": 10}
    income = {"Less than 3 Lakh:":2, "3 - 6 Lakh:":4 ," 6 - 10 Lakh:": 6," 10 - 20 Lakh:": 8 ,"Greater than 20 Lakh:": 10}
    education_levels={"No Formal Education":2,"High School":4, "Associate Degree":4, "Bachelor's Degree":4, "Master's Degree":6, "PhD/ Finance Certification": 8}
    risk_tolerance_ratio = (age_factor[user_age]*0.2 + investment_exp_factor[user_investment_exp]*0.3 + income[user_income]*0.3 + education_levels[user_education]*0.2)
    return risk_tolerance_ratio

# Streamlit UI
st.title("Finance Dictionary Chatbot")
st.write("Ask about finance terms, and I'll give you both a formal definition and a simple explanation.")

# User input
user_query = st.text_input("Enter a finance term:", key="user_query")


# Dropdown for Age
ages = ["18-25", "26-30", "31-40", "41-50", "51-60", "60+"]  # You can modify the age ranges or options as needed
user_age = st.selectbox("Select your age group:", ages)

# Dropdown for Education
education_levels = ["No Formal Education","High School", "Associate Degree", "Bachelor's Degree", "Master's Degree", "PhD"]
user_education = st.selectbox("Select your highest education level:", education_levels)

# Dropdown for Investment Experience
investment_experience = ["Less than 1 year", "1-3 years", "3-5 years", "5+ years"]
user_investment_exp = st.selectbox("Select your investment experience:", investment_experience)

# Dropdown for Investment Amount
income = ["Less than 3 Lakh:", "3 - 6 Lakh:"," 6 - 10 Lakh:"," 10 - 20 Lakh:","Greater than 20 Lakh:"]
user_income = st.selectbox("Select the amount you are willing to invest:", income)

# # Dropdown for Willing Risk
# investment_amounts = ["Less than 1000", "$1,000 - $5,000", "$5,000 - $10,000", "$10,000+"]
# user_investment_amt = st.selectbox("Select the amount you are willing to invest:", investment_amounts)

# Dropdown for Willingness to Take Risk
risk_levels = ["Low", "Medium", "High"]
user_risk = st.selectbox("Select your willingness to take risk:", risk_levels)

# Dropdown for Investment Goals
investment_goals = ["Retirement", "Wealth Accumulation", "Education Fund", "Buying a Home", "Short-Term Gains", "Other"]
user_investment_goal = st.selectbox("Select your investment goal:", investment_goals)


if st.button("Explain", key="explain_button"):
    if user_query:
        finance_dict = load_finance_dictionary()
        finance_dict = {key.lower(): value for key, value in finance_dict.items()}

        term = user_query.strip().lower()

        # Check if term exists in dictionary
        if term in finance_dict:
            definition = finance_dict[term]
            simple_explanation = generate_simple_explanation(term)
            
            # Display results
            st.markdown(f"### **Definition:**\n{definition}")
            st.markdown(f"### **Simple Explanation:**\n{simple_explanation}")

        else:
            simple_explanation = generate_simple_explanation(term)
            proper_explanation = generate_proper_explanation(term)
            
            if proper_explanation:
            # Display results
                st.markdown(f"### **Definition:**\n{proper_explanation}")
                st.markdown(f"### **Simple Explanation:**\n{simple_explanation}")
                
                with open("finance_terms.json", "r", encoding="utf-8") as file:
                    data_json = json.load(file)
                
                data_json[term] = proper_explanation
                
                with open("finance_terms.json", "w") as file:
                    json.dump(data_json, file, indent=4, ensure_ascii=False)
                            
            else:
                st.error("Sorry, I couldn't explain this term. I will update my dictionary. Try another one!")
    else:
        st.error("Please enter a finance term.")
